# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

In addition to implementing code, there is a brief writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) that can be used to guide the writing process. Completing both the code in the Ipython notebook and the writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/322/view) for this project.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="examples/line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="examples/laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, see [this forum post](https://carnd-forums.udacity.com/cq/viewquestion.action?spaceKey=CAR&id=29496372&questionTitle=finding-lanes---import-cv2-fails-even-though-python-in-the-terminal-window-has-no-problem-with-import-cv2) for more troubleshooting tips.**  

## Import Packages

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

## Read in an Image

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')

#printing out some stats and plotting
print('This image is:', type(image), 'with dimensions:', image.shape)
plt.imshow(image)  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')

## Ideas for Lane Detection Pipeline

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

## Helper Functions

Below are some helper functions to help get you started. They should look familiar from the lesson!

## Test Images

Build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
import os
os.listdir("test_images/")

In [ ]:
import math
import statistics

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def line_length(x1, y1, x2, y2):
    x = x1 - x2
    y = y1 - y2
    return math.sqrt((x*x)+(y*y))

def line_slope(x1, y1, x2, y2):
    if x2 == x1:
        return 0
    
    return (y2-y1)/(x2-x1)

# courtesy https://stackoverflow.com/questions/20677795/how-do-i-compute-the-intersection-point-of-two-lines-in-python
def line_intersection(line1, line2):
    xdiff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
    ydiff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1]) #Typo was here

    def det(a, b):
        return a[0] * b[1] - a[1] * b[0]

    div = det(xdiff, ydiff)
    if div == 0:
       raise Exception('lines do not intersect')

    d = (det(*line1), det(*line2))
    x = det(d, xdiff) / div
    y = det(d, ydiff) / div
    return x, y

# This routine can optionally take in previously found
# left/right vertices on same image - it will then not
# re-draw the left/right segments.
def draw_lines(img, lines, color=[255, 0, 0], thickness=2,
               left_vertices=None, right_vertices=None):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """    
    from operator import itemgetter
    
    # stores all detected segments from HoughLinesP alongwith slope, intercept and length
    all_seg_info = []
    for line in lines:
        for x1,y1,x2,y2 in line:
            slope = (y2-y1)/(x2-x1)
            if (slope < 0) and left_vertices:
                # left already handled in earlier pass, ignore segment
                continue
            if (slope > 0) and right_vertices:
                # right already handled in earlier pass, ignore segment
                continue
            intercept = y1 - (slope*x1)
            length = line_length(x1,y1,x2,y2)
                
            all_seg_info.append(
                (x1, y1, x2, y2, slope, intercept, length))
            
            # Debug for seeing 'raw' segments
            #cv2.line(img, (x1, y1), (x2, y2), color, thickness)
    
    # These are the parameters to determine if segment is valid
    # and if valid whether it is left or right portion.
    left_slope_min = -0.85
    left_slope_max = -0.65
    right_slope_min = 0.5
    right_slope_max = 0.7
    
    # classify to left/right + sort by line segment's y1 (so bottom to top of img)
    if left_vertices:
        # create a segment from previous pass
        bottom_left_x, bottom_left_y, top_left_x, top_left_y = left_vertices
        slope = (top_left_y-bottom_left_y)/(top_left_x-bottom_left_x)
        intercept = bottom_left_y - (slope*bottom_left_x)
        length = line_length(bottom_left_x, bottom_left_y,
                             top_left_x, top_left_y)
        left_seg_info = [(bottom_left_x, bottom_left_y, top_left_x, top_left_y,
                          slope, intercept, length)]
    else:
        left_seg_info = sorted([si for si in all_seg_info
                                    if (left_slope_min<=si[4]<=left_slope_max)],
                               key=itemgetter(1),
                               reverse=True)
    if right_vertices:
        # create a segment from previous pass
        bottom_right_x, bottom_right_y, top_right_x, top_right_y = right_vertices
        slope = (top_right_y-bottom_right_y)/(top_right_x-bottom_right_x)
        intercept = bottom_right_y - (slope*bottom_right_x)
        length = line_length(bottom_right_x, bottom_right_y,
                             top_right_x, top_right_y)
        right_seg_info = [(bottom_right_x, bottom_right_y, top_right_x, top_right_y,
                          slope, intercept, length)]

    else:
        right_seg_info = sorted([si for si in all_seg_info
                                    if (right_slope_min<=si[4]<=right_slope_max)],
                                key=itemgetter(1),
                                reverse=True)
    
    # avg is really median..
    avg_left_slope = statistics.median([si[4] for si in left_seg_info] or [0])
    avg_right_slope = statistics.median([si[4] for si in right_seg_info] or [0])
    avg_left_int = statistics.median([si[5] for si in left_seg_info] or [0])
    avg_right_int = statistics.median([si[5] for si in right_seg_info] or [0])
    
    # filter out outlier segments w.r.t slope/intersection
    left_seg_info = [si for si in left_seg_info
                        if ((np.abs(si[4]-avg_left_slope) <= 0.05*np.abs(avg_left_slope)) and
                             (np.abs(si[5]-avg_left_int) <= 0.03*np.abs(avg_left_int)))]
    right_seg_info = [si for si in right_seg_info
                         if ((np.abs(si[4]-avg_right_slope) <= 0.05*np.abs(avg_right_slope)) and
                             (np.abs(si[5]-avg_right_int) <= 2*np.abs(avg_right_int)))]

    
    # Create polylines to draw on original image.
    left_polylines = []
    right_polylines = []
    
    # calculate valid lengths for later reference to see if at bottom of image
    # width between left and right segments are too large/small
    valid_left_len = valid_right_len = 0
    
    # filter if slope differs by more than 20% of median
    for x1,y1,x2,y2,slope,intercept,length in left_seg_info:
        #print("got x1,y1,inter,len from left_seg_info ", x1, y1, intercept, length)

        if left_polylines:
            # ignore a point that differs much from avg slope
            slope = line_slope(left_polylines[-1][0], left_polylines[-1][1],
                               x1, y1)
            if np.abs(slope - avg_left_slope) > np.abs(0.2*avg_left_slope):
                continue
        valid_left_len += length
        left_polylines.append([x1,y1])
        
        # Debug for seeing filtered 'raw' segments
        #cv2.line(img, (x1,y1), (x2,y2), [0,255,0], thickness)

    # filter if slope differs by more than 20% of median
    for x1,y1,x2,y2,slope,intercept,length in right_seg_info:
        #print("got x1,y1,inter,slope,len from right_seg_info ", x1, y1, intercept, slope, length)

        if right_polylines:
            # ignore a point that differs much from avg slope
            slope = line_slope(right_polylines[-1][0], right_polylines[-1][1],
                               x1, y1)
            if np.abs(slope - avg_right_slope) > np.abs(0.2*avg_right_slope):
                #print("ignoring ", x1, y1)
                continue
        #print("adding to right-poly ", x1, y1)
        valid_right_len += length
        right_polylines.append([x1,y1])

        # Debug for seeing filtered 'raw' segments
        #cv2.line(img, (x1,y1), (x2,y2), [0,255,0], thickness)
        

    # Used to extrapolate down and up.
    left_first = (left_seg_info or [0])[0]
    left_last = (left_seg_info or [0])[-1]
    right_first = (right_seg_info or [0])[0]
    right_last = (right_seg_info or [0])[-1]

    # extrapolate to bottom of image
    bottom_y = img.shape[0]
    if left_first:
        bottom_left_x = int((bottom_y - avg_left_int)/avg_left_slope)
        left_polylines.insert(0, (bottom_left_x, bottom_y))
        
    if right_first:
        bottom_right_x = int((bottom_y - avg_right_int)/avg_right_slope)
        right_polylines.insert(0, (bottom_right_x, bottom_y))
    
    if left_first and right_first:
        # verify bottom interpolation is within range,
        # else omit right/left based on the greater valid-length
        bottom_width = line_length(left_polylines[0][0], left_polylines[0][1],
                               right_polylines[0][0], right_polylines[0][1])
        bottom_x_ratio = bottom_width/img.shape[1]
        #print("bottom_x_ratio ", bottom_x_ratio)
        if (bottom_x_ratio <= 0.7) or (bottom_x_ratio >= 0.77):
            # one of the segments is wrongly thought to be lane
            #print("valid left/right ", valid_left_len, valid_right_len)
            if valid_left_len > valid_right_len:
                right_polylines = []
                right_first = right_last = None
                valid_right_len = 0
            else:
                left_polylines = []
                left_first = left_last = None
                valid_left_len = 0
                
        
    # extrapolate to top of image
    # 1. find intersection 2. clip points beyond 3. extrapolate upto
    if left_last and right_last:
        intersection_pt = line_intersection([[left_last[0], left_last[1]], [left_last[2], left_last[3]]],
                                            [[right_last[0], right_last[1]], [right_last[2], right_last[3]]])
    else:
        intersection_pt = None

    if intersection_pt:
        # clip segments beyond intersection point
        left_polylines = [l for l in left_polylines 
                            if (l[0] <= intersection_pt[0]) and (l[1] >= intersection_pt[1])]
        right_polylines = [l for l in right_polylines 
                             if (l[0] >= intersection_pt[0]) and (l[1] >= intersection_pt[1])]

        # extend upto intersection point's 'y' (- delta)...
        top_y = intersection_pt[1] + 10
        top_left_x = int((top_y - avg_left_int)/avg_left_slope)                                
        top_right_x = int((top_y - avg_right_int)/avg_right_slope)
        
        # ...and clip back if crossed-over
        if top_left_x > top_right_x:
            top_x = int((top_left_x+top_right_x)/2) + 25
            top_left_y = int((top_x*avg_left_slope) + avg_left_int)
            top_right_y = int((top_x*avg_right_slope) + avg_right_int)
            left_polylines.append((top_x, top_left_y))
            right_polylines.append((top_x, top_right_y))
        else:
            left_polylines.append((top_left_x, top_y))
            right_polylines.append((top_right_x, top_y))

    if left_vertices:
        # nothing changed, don't draw
        pass
    elif left_polylines:
        # sort lines from bottom to top
        left_polylines = sorted(left_polylines, key=itemgetter(1), reverse=True)
        left_pts = np.array([x for x in left_polylines], np.int32)
        bottom_left_x = left_polylines[0][0]
        bottom_left_y = left_polylines[0][1]
        top_left_x = left_polylines[-1][0]
        top_left_y = left_polylines[-1][1]
        cv2.polylines(img, [left_pts], False, [0,0,255], thickness)
    else:
        # no luck finding any segment
        bottom_left_x = bottom_left_y = top_left_x = top_left_y = 0
    
    if right_vertices:
        # nothing changed, don't draw
        pass
    elif right_polylines:
        # sort lines from bottom to top
        right_polylines = sorted(right_polylines, key=itemgetter(1), reverse=True)
        right_pts = np.array([x for x in right_polylines], np.int32)
        bottom_right_x = right_polylines[0][0]
        bottom_right_y = right_polylines[0][1]
        top_right_x = right_polylines[-1][0]
        top_right_y = right_polylines[-1][1]
        cv2.polylines(img, [right_pts], False, [255,0,0], thickness)
    else:
        # no luck finding any segment
        bottom_right_x = bottom_right_y = top_right_x = top_right_y = 0
            
    return ((bottom_left_x, bottom_left_y, top_left_x, top_left_y),
            (bottom_right_x, bottom_right_y, top_right_x, top_right_y),
            valid_left_len, valid_right_len)
    

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap, left_vertices=None, right_vertices=None):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    left_vertices, right_vertices, left_len, right_len = draw_lines(
        line_img, lines, thickness=5, left_vertices=left_vertices, right_vertices=right_vertices)

    return line_img, left_vertices, right_vertices, left_len, right_len

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Build a Lane Finding Pipeline



Build the pipeline and run your solution on all test_images. Make copies into the `test_images_output` directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

In [ ]:
# Helper routine to debug/view.
def draw_test_line(file_name, x1, y1, slope=None, x2=None, y2=None):
    my_image = mpimg.imread(file_name)
    print('x = , y = ', my_image.shape[1], my_image.shape[0])
    if slope is not None:   
        intercept = y1 - (slope*x1)
        y2 = 270
        x2 = (y2 - intercept)/slope
    else:
        intercept = None
    print(x1, y1, x2, y2, slope, intercept)
    cv2.line(my_image, (int(x1),int(y1)), (int(x2),int(y2)), [0,255,0], 10)
    plt.subplot(1,1,1)
    plt.imshow(my_image)
    plt.xticks([]),plt.yticks([])

# This routine processes a given grayscale image
# (for e.g. produced by RGB2GRAYSCALE or HSV2GRAYSCALE)
# Has most of the pipeline for processing.
def process_grayscale_image(gs_image, orig_image, left_vertices=None, right_vertices=None):
    gb_image = gaussian_blur(gs_image, 5)
    lower_thresh = 50
    upper_thresh = 150
    canny_image = canny(gb_image, lower_thresh, upper_thresh)
    if left_vertices and right_vertices:
        roi_vertices = np.array([[
            (int(left_vertices[0]), int(left_vertices[1])),
            (int(left_vertices[2]), int(left_vertices[3])),
            (int(right_vertices[0]), int(right_vertices[1])),
            (int(right_vertices[2]), int(right_vertices[3])),        
        ]])
    else:
        roi_vertices =  np.array([[
            (int(orig_image.shape[1]*0.15), orig_image.shape[0]),
            (int(orig_image.shape[1]*0.45), int(orig_image.shape[0]*0.5)),
            (int(orig_image.shape[1]*0.55), int(orig_image.shape[0]*0.5)),
            (int(orig_image.shape[1]*0.9), orig_image.shape[0])
        ]])

    
    roi_image = region_of_interest(canny_image, roi_vertices)
    
    thresh = 15
    min_line_length = 15
    max_line_gap = 5
    lines_image, left_vertices, right_vertices, left_len, right_len = hough_lines(
        roi_image, 1, np.pi/180, thresh, min_line_length, max_line_gap,
        left_vertices=left_vertices, right_vertices=right_vertices)
    weighted_image = weighted_img(lines_image, orig_image)
    
    #plt.subplot(1,1,1)
    #plt.imshow(weighted_image)
    #plt.xticks([]),plt.yticks([])
    #plt.show()

    return weighted_image, left_vertices, right_vertices, left_len, right_len
   

# The main pipeline handler, tries rgb2grayscale and if both segments not found,
# hsv2grayscale (It remembers previous processing so hsv2grayscale is only for
# segment that had trouble in detection)
def process_test_image(image=None, file_name=None, out_file_name=None):
    if image is not None:
        img = image
    elif file_name is not None:        
        img = mpimg.imread(file_name)


    weighted_image, left_vertices, right_vertices, left_len, right_len = process_grayscale_image(
        grayscale(img), img[:,:])
    
    if (left_len == 0) or (right_len == 0):
        #print('rgb left,right ', left_len, right_len, right_vertices)
        # try with HSV->GS
        if (left_len == 0) and (right_len != 0):
            weighted_image, left_vertices, right_vertices, left_len, right_len = process_grayscale_image(
                grayscale(cv2.cvtColor(img, cv2.COLOR_BGR2HSV)), 
                weighted_image, right_vertices=right_vertices)
        elif (right_len == 0) and (left_len != 0):
            weighted_image, left_vertices, right_vertices, left_len, right_len = process_grayscale_image(
                grayscale(cv2.cvtColor(img, cv2.COLOR_BGR2HSV)), 
                weighted_image, left_vertices=left_vertices)

        #print('hsv left,right ', left_len, right_len)
    
    if out_file_name:
        cv2.imwrite(out_file_name, weighted_image)
    return weighted_image


output_dir = 'test_images_output/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
for file in os.listdir("test_images/"):
   process_test_image(
        file_name='test_images/'+file,
        out_file_name='test_images_output/'+file)

#draw_test_line('test_images/solidWhiteCurve.jpg', 890, 540, 0.553)
#draw_test_line('test_images/solidWhiteCurve.jpg', 890, 540, x2=401, y2=270)
#draw_test_line('test_images/solidYellowCurve.jpg', 840, 540, 0.75)
#draw_test_line('test_images/solidYellowCurve.jpg', 170, 540, -0.75)

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an `import error` when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt. Also, check out [this forum post](https://carnd-forums.udacity.com/questions/22677062/answers/22677109) for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://carnd-forums.udacity.com/display/CAR/questions/26218840/import-videofileclip-error) for more troubleshooting tips across operating systems.**

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(image):
    result = process_test_image(image)
    return result

Let's try the one with the solid white lane on the right first ...

In [ ]:
output_dir = 'test_videos_output/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
#clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(1,2)
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Improve the draw_lines() function

**At this point, if you were successful with making the pipeline and tuning parameters, you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform. As mentioned previously, try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines. You can see an example of the result you're going for in the video "P1_example.mp4".**

**Go back and modify your draw_lines function accordingly and try re-running your pipeline. The new output should draw a single, solid line over the left lane line and a single, solid line over the right lane line. The lines should start from the bottom of the image and extend out to the top of the region of interest.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
#clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(21.5,21.75)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Writeup and Submission

If you're satisfied with your video outputs, it's time to make the report writeup in a pdf or markdown file. Once you have this Ipython notebook ready along with the writeup, it's time to submit for review! Here is a [link](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) to the writeup template file.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'test_videos_output/challenge.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds#
#clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(3,3.5)
clip3 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip3.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))